In [1]:
import json
import sys
import signal
import os.path
import argparse
import datetime
import librosa
import numpy as np

input_dir=''
output_dir=''

In [4]:

def signal_handler(signum, frame):
    print('Interrupted')
    sys.exit(0)

# Handle C-c
signal.signal(signal.SIGINT, signal_handler)

##############
#  Analysis  #
##############
def analyze(files):
  """
  Audio feature extraction
  """
  output = []

  for file in files:
    # Load the audio as a waveform `y`
    # Store the sampling rate as `sr`
    y, sr = librosa.load(file)

    # spectral centroid
    spectral_centroids = librosa.feature.spectral_centroid(y=y, sr=sr)
    # spectral bandwidth
    spectral_bandwidths = librosa.feature.spectral_bandwidth(y=y, sr=sr)
    # mel-frequency cepstral coefficients
    S = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128)
    mfccs = librosa.feature.spectral.mfcc(y=y, sr=sr, S=librosa.core.logamplitude(S), n_mfcc=12)
    # duration
    duration = librosa.core.get_duration(y=y, sr=sr)

    json_data = {
      'path': file,
      'duration': duration,
      'spectral_centroids': spectral_centroids[0].tolist(),
      'spectral_bandwidths': spectral_bandwidths[0].tolist(),
      'spectral_centroid_min': spectral_centroids.min(),
      'spectral_centroid_max': spectral_centroids.max(),
      'spectral_centroid_mean': spectral_centroids.mean(),
      'mfccs': mfccs.tolist(),
      'mfcc_mean': mfccs.mean(),
    }

    print('Analyzing file:', file)
    output.append(json_data)

  return output

In [8]:

# Get paths for audio files
valid_extensions = ['aac', 'au', 'flac', 'm4a', 'mp3', 'ogg', 'wav', 'aif']
audio_file_path = os.path.expanduser(input_dir)

if os.path.isdir(audio_file_path):
  audio_files = librosa.util.find_files(audio_file_path,
                                        ext=valid_extensions)
else:
  audio_files = [ audio_file_path ]

# do the analysis
result = analyze(audio_files)

#########
#  I/O  #
#########
def parse_input():
  """
  Parse user input to determine if we should overwrite the existing file
  """
  # raw_input returns the empty string for "enter"
  yes = set(['yes','y', 'ye', ''])
  no = set(['no','n'])
  choice = input('-> ').lower()

  while True:
    if choice in yes:
      return True
    elif choice in no:
      return False
    else:
      sys.stdout.write("Please respond with 'yes' or 'no'")

/home/dexter/anaconda3/lib/python3.6/site-packages/librosa/core/audio.py:146: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')


FileNotFoundError: [Errno 2] No such file or directory: ''

In [9]:
def write_file(path, data):
  """
  Write data to file
  """
  with open(path, 'w') as file:
    json.dump(data, file, separators=(',', ':'))

  print('Wrote output to', path)

if output_dir:
  json_path = os.path.expanduser(output_dir)
else:
  json_path = os.path.abspath('./output.json')

# json document structure
json_output = {
  'files': result,
  'timestamp': datetime.datetime.now().isoformat(),
  'version': '0.1'
}

if os.path.isfile(json_path):
  print('File {0} exists'.format(json_path))
  print('Overwrite?\n y/n')
  overwrite = parse_input()
  if overwrite:
    write_file(json_path, json_output)
  else:
    print('File was not overwritten')
else:
  write_file(json_path, json_output)

NameError: name 'result' is not defined